[GR00T N1: An Open Foundation Model for Generalist Humanoid Robots](https://arxiv.org/abs/2503.14734)

**Abstract**  
General-purpose robots need a versatile body and an intelligent mind. Recent advancements in humanoid robots have shown great promise as a hardware platform for building generalist autonomy in the human world. A robot foundation model, trained on massive and diverse data sources, is essential for enabling the robots to reason about novel situations, robustly handle real-world variability, and rapidly learn new tasks. To this end, we introduce GR00T N1, an open foundation model for humanoid robots. GR00T N1 is a Vision-Language-Action (VLA) model with a dual-system architecture. The vision-language module (System 2) interprets the environment through vision and language instructions. The subsequent diffusion transformer module (System 1) generates fluid motor actions in real time. Both modules are tightly coupled and jointly trained end-to-end. We train GR00T N1 with a heterogeneous mixture of real-robot trajectories, human videos, and synthetically generated datasets. We show that our generalist robot model GR00T N1 outperforms the state-of-the-art imitation learning baselines on standard simulation benchmarks across multiple robot embodiments. Furthermore, we deploy our model on the Fourier GR-1 humanoid robot for language-conditioned bimanual manipulation tasks, achieving strong performance with high data efficiency.


# GR00T N1 Foundation Model
GR00T N1 is a Vision-Language-Action (VLA) model for humanoid robots trained on diverse data sources. The model contains a vision-language backbone that encodes language and image input and a DiT-based flow-matching policy that outputs high-frequency actions. We use the NVIDIA Eagle-2 VLM (Li et al., 2025) as the vision-language backbone. Specifically, our publicly released GR00T-N1-2B model has 2.2B parameters in total, with 1.34B in the VLM. The inference time for sampling a chunk of 16 actions is 63.9ms on an L40 GPU using bf16. Fig. 2 provides a high-level overview of our model design. We highlight three key features of GR00T N1:
- We design a compositional model that integrates Vision-Language Model (VLM)-based reasoning module (System 2) and Diffusion Transformer (DiT)-based action module (System 1) in a unified learning framework;
- We develop an effective pre-training strategy using a mixture of human videos, simulation and neural-generated data, and real robot demonstrations (see Fig. 1) for generalization and robustness;
- We train a massively multi-task, language-conditioned policy that supports a wide range of robot embodiments and enables rapid adaptation to new tasks through data-efficient post-training.

GR00T N1是一个在不同数据源上训练的人形机器人的 Vision-Language-Action (VLA) 模型。该模型包含一个视觉-语言骨干(编码语言和图像输入)以及基于 DIT 的 flow-matching 策略(输出高频工作)。我们使用 NVIDIA Eagle-2 VLM (Li et al., 2025) 作为视觉-语言骨干。具体而言, 我们公开发布的 GR00T-N1-2B 模型总共有 2.2b 参数, 其中 VLM 中的参数为1.34B。使用BF16，在L40 GPU上抽样16个动作的推理时间为63.9ms。图2提供了我们模型设计的高级概述。我们重点介绍了GR00T N1的三个关键功能：
 - 我们设计了一个组成模型，该模型将基于视觉语言模型（VLM）的推理模块（系统2）和扩散变压器（DIT）基于统一的学习模块（系统1）在统一的学习框架中；
 - 我们使用人类视频，仿真和神经生成的数据以及真正的机器人演示的混合物制定了有效的训练策略（请参见图1）；
 - 我们训练一项大规模的多任务，语言条件政策，该政策支持广泛的机器人实施例，并通过数据有效的后培训来快速适应新任务。

GR00T N1 是一个针对人形机器人的视觉-语言-动作（VLA）模型，训练于多种数据源。该模型包含一个编码语言和图像输入的视觉-语言骨干，以及一个基于扩散变换器（DiT）的流匹配策略，用于输出高频率的动作。我们使用 NVIDIA Eagle-2 VLM（Li et al., 2025）作为视觉-语言骨干。具体来说，我们公开发布的 GR00T-N1-2B 模型总共有 22 亿个参数，其中 13.4 亿个参数在 VLM 中。采样 16 个动作的推理时间为 63.9 毫秒，使用 bf16 在 L40 GPU 上进行。图 2 提供了我们模型设计的高层次概述。我们强调 GR00T N1 的三个关键特性：

我们设计了一个组合模型，集成了基于视觉-语言模型（VLM）的推理模块（系统 2）和基于扩散变换器（DiT）的动作模块（系统 1），形成统一的学习框架；
我们开发了一种有效的预训练策略，利用人类视频、仿真和神经生成数据以及真实机器人演示的混合，增强模型的泛化能力和鲁棒性（见图 1）；
我们训练了一个大规模的多任务、语言条件的策略，支持多种机器人形态，并通过数据高效的后训练实现快速适应新任务。

## Model Architecture
In this section, we describe the GR00T N1 model architecture, illustrated in Fig. 3. GR00T N1 uses flow-matching (Lipman et al.) to learn action generation. A diffusion transformer (DiT) processes the robot’s proprioceptive state and action, which are then cross-attended with image and text tokens from the Eagle-2 VLM backbone to output the denoised motor actions. Below, we elaborate on each module in detail.

Figure 3: **GR00T N1 Model Architecture**. GR00T N1 is trained on a diverse set of embodiments ranging from si gle-arm robot arms to bimanual humanoid dexterous hands. To deal with different robot embodiment’s state observation and action, we use DiT blocks with an embodiment-aware state and action encoder to embed the robot’s state and action inputs. GR00T N1 model leverages latent embeddings of the Eagle-2 model to incorporate the robot’s visual observation and language instructions. The vision language tokens will then be fed into the DiT blocks through cross-attention layers.

**State and Action Encoders**

To process states and actions of varying dimensions across different robot embodiments, we use an MLP per embodiment to project them to a shared embedding dimension as input to the DiT. As in Black et al. (2024), the Action Encoder MLP also encodes the diffusion timestep together with the noised action vector.

We use action flow matching, which samples actions through iterative denoising. The model takes as input noised actions in addition to encodings of the robot’s proprioceptive state, image tokens, and text tokens. The actions are processed in chunks as in Zhao et al. (2023), meaning that at any given time 𝑡the model uses $𝐴_𝑡 = [𝑎_𝑡, 𝑎_{𝑡+1}, \dots, 𝑎_{𝑡+𝐻−1}]$ which contains the action vectors of timesteps $𝑡$ through $𝑡+𝐻−1$. We set $𝐻= 16$ in our implementation.

**Vision-Language Module (System 2)**

For encoding vision and language inputs, GR00T N1 uses the Eagle-2 (Li et al., 2025) vision-language model (VLM) pretrained on Internet-scale data. Eagle-2 is finetuned from a SmolLM2 (Allal et al., 2025) LLM and a SigLIP-2 (Tschannen et al., 2025) image encoder. Images are encoded at resolution 224 $\times$ 224followed by pixel shuffle (Shi et al., 2016), resulting in 64 image token embeddings per frame. These embeddings are then further encoded together with text by the LLM component of the Eagle-2 VLM. The LLM and image encoder are aligned over a broad set of vision-language tasks following the general recipe of Li et al. (2025).

During policy training, a text description of the task, as well as (possibly multiple) images, are passed to the VLM in the chat format used during vision-language training. We then extract vision-language features of shape (batch size $\times$ sequence length $\times$ hidden dimension) from the LLM. We found that using middle-layer instead of final-layer LLM embeddings resulted in both faster inference speed and higher downstream policy success rate. For GR00T-N1-2B, we use the representations from the 12th layer.

**Diffusion Transformer Module (System 1)**

For modeling actions, GR00T N1 uses a variant of DiT (Peebles and Xie, 2023), which is a transformer with denoising step conditioning via adaptive layer normalization, denoted as $𝑉_{\theta}$. As shown in Fig. 3, $𝑉_{\theta}$ consists of


In [7]:
from pypdf import PdfReader

pdf_path = "/mnt/z/AutoPapers/downloaded_papers/2503.14734.GR00T_N1_An_Open_Foundation_Model_for_Generalist_Humanoid_Robots.pdf"

reader = PdfReader(pdf_path)
number_of_pages = len(reader.pages)
page = reader.pages[3]
text = page.extract_text()
print(text)

GR00T N1: An Open Foundation Model for Generalist Humanoid Robots
Robot State
Eagle-2 VLMCross-AttentionSelf-AttentionCross-AttentionSelf-AttentionStateEncoderAction DecoderDiTBlocks MotorActionAction Encoderx N“Pick up the apple and place it into the bottom shelf”
Noised ActionEmbodiment-Specific ModuleVisionEncoderText Tokenizer
K iterations
Pre-trained and Frozen𝑞!. . .𝑎!𝑎!"#𝑎!"$%#. . .𝑎!𝑎!"#𝑎!"$%#
Figure 3: GR00T N1 Model Architecture. GR00T N1 is trained on a diverse set of embodiments ranging from
single-arm robot arms to bimanual humanoid dexterous hands. To deal with different robot embodiment’s
state observation and action, we use DiT blocks with an embodiment-aware state and action encoder to embed
the robot’s state and action inputs. GR00T N1 model leverages latent embeddings of the Eagle-2 model to
incorporate the robot’s visual observation and language instructions. The vision language tokens will then be
fed into the DiT blocks through cross-attention layers.
the Action E